# Use Case for Agentic RAG

## Introduction to Agentic RAG

LLMs are trained on enormous bodies of data to learn general knowledge. However, the world knowledge model of LLMs may not always be relevant and up-to-date information. RAG solves this problem by finding and retrieving relevant information from our data and forwarding that to the LLM. Agentic RAG is a powerful way to use agents to answer questions about our data.

Each step will be implemented using `smolagents`, `llama-index`, and `langgraph`, respectively.

## Creating a RAG Tool for Guest Stories

We will implement an agent within a HF Space, as a structured Python project..

Project structure
- `tools.py` - provides auxiliary tools for the agent
- `retriever.py` - implements retrieval functions to support knowledge access
- `app.py` - integrates all components into a fully functional agent.

The complete development is live [HERE](https://huggingface.co/spaces/agents-course/Unit_3_Agentic_RAG)

### Dataset

Our dataset as external knowledge base [`agents-course/unit3-invitees`](https://huggingface.co/datasets/agents-course/unit3-invitees) contains
- Name - Guest's full name
- Relation - How the guest is related to the host
- Description - A brief biography or interesting facts about the guest
- Email address - Contact information for sending invitations or follow-ups

### Building the Guestbook Tool

We will create a custom tool that the agent can use to quickly retrieve guest information. There are 3 steps to complete:
1. Load and prepare the dataset
2. Create the Retriever Tool
3. Integrate the Tool with agent

#### Step 1 - load and prepare the dataset

We need to transform our raw guest data into a format that is optimized for retrieval.

`smolagent`

In [ ]:
import datasets
from langchain.docstore.document import Document

# Load the dataset
guest_dataset = datasets.load_dataset(
    'agents-course/unit3-invitees',
    split='train'
)

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content='\n'.join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={'name': guest['name']}
    )
    for guest in guest_dataset
]

`llama-index`

In [ ]:
import datasets
from llama_index.core.schema import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

`langgraph`

In [ ]:
import datasets
from langchain.docstore.document import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

#### Step 2 - create the Retriever tool

`smolagents`

In [ ]:
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever

class GuestInfoRetrieverTool(Tool):
    name = "guest_info_retriever"
    description = "Retrieves detailed information about gala guests based on their name or relation."
    inputs = {
        'query': {
            'type': 'string',
            'description': 'The name or relation of the guest you want information about.'
        }
    }
    output_type = 'string'

    def __init__(self, docs):
        self.is_initialized = False
        self.retriever = BM25Retriever.from_documents(docs)

    def forward(self, query: str):
        results = self.retriever.get_relevant_documents(query)
        if results:
            return "\n\n".join([doc.page_content for doc in reuslts[:3]])
        else:
            return "No matching guest information found."


# Initialize the tool
guest_info_tool = GuestInfoRetrieverTool(docs)

`llama-index`

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.retrievers.bm25 import BM25Retriever

bm25_retriever = BM25Retriever.from_defaults(nodes=docs)

def get_guest_info_retriever(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.retrieve(query)
    if results:
        return '\n\n'.join([doc.text for doc in results[:3]])
    else:
        return 'No matching guest information found.'

# Initialize the tool
guest_info_tool = FunctionTool.from_defaults(get_guest_info_retriever)

`langgraph`

In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool

bm25_retriever = BM25Retriever.from_documents(docs)

def extract_text(query: str) -> str:
    """Retreives detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.invoke(query)
    if results:
        return '\n\n'.join([doc.page_content for doc in results[:3]])
    else:
        return 'No matching guest information found.'


# Initialzie the tool
guest_info_tool = Tool(
    func=extract_text,
    name='guest_info_retriever',
    description='Retrieves detailed information about gala guests based on their name or relation.'
)

#### Step 3 - integrate the tool with agent

`smolagents`

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

# Initialize the HuggingFace model
model = InferenceClientModel()

# Create agent
agent = CodeAgent(
    model=model,
    tools=[guest_info_tool]
)

# Test example
response = agent.run(
    "Tell me about our guest named 'Lady Ada Lovelace'."
)
print("Agent's Response:")
print(repsonse)

`llama-index`

In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

# Initialize the HuggingFace model
llm = HuggingFaceInferenceAPI(model_name='Qwen/Qwen2.5-Coder-32B-Instruct')

# Create agent
agent = AgentWorkflow.from_tools_or_functions(
    [guest_info_tool],
    llm=llm
)

# Test example
response = agent.run(
    "Tell me about our guest named 'Lady Ada Lovelace'."
)
print("Agent's Response:")
print(repsonse)

`langgraph`

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import START, StateGraph
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

from google.colab import userdata
HFHUB_API_TOKEN = userdata.get('HF_TOKEN')

# Initialize the HuggingFace model

# Generate the chat interface, including the tools
llm = HuggingFaceEndpoint(
    repo_id='Qwen/Qwen2.5-Coder-32B-Instruct',
    huggingfacehub_api=HUGGINGFACEHUB_API_TOKEN
)

chat = ChatHuggingFace(llm=llm, verbose=True)
tools = [guest_info_tool]
chat_with_tools = chat.bind_tools(tools)

# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


def assistant(state: AgentState):
    return {
        'messages': [chat_with_tools.invoke(state['messages'])]
    }




# The graph
builder = StateGraph(AgentState)

# Define nodes
builder.add_node('assistant', assistant)
builder.add_node('tools', ToolNode(tools))

# Define edges
builder.add_edge(START, 'assistant')
builder.add_conditional_edges(
    'assistant',
    # If the latest message requires a tool, route to tools,
    # Otherwise, provide a direct response
    tools_condition
)
builder.add_edge('tools', 'assistant')

agent = builder.compile()


# Test example
messages = [
    HumanMessage(content='Tell me about our guest named "Lady Ada Lovelace".')
]
response = agent.invoke({'messages': messages})
print("Agent's Response:")
print(response['messages'][-1].content)

## Building and Integrating Tools for Agent

Then, we will grant the agent access to the web, enabling it to find the latest news and global updates. In addition, the agent will have access to weather data and HuggingFace hub model download statistics, so that it can make relevant conversation about fresh topics.

### Give agent access to the web

`smolagents`

In [ ]:
from smolagents import DuckDuckGoSearchTool

# Initialize the DuckDuckGo search tool
search_tool = DuckDuckGoSearchTool()

# Example test
results = search_tool("Who's the current General Manager of Golden State Warriors?")
print(results)

`llama-index`

In [ ]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.core.tools import FunctionTool

# Initialize the DuckDuckGo search tool
tool_spec = DuckDuckGoSearchToolSpec()
search_tool = FunctionTool.from_defaults(tool_spec.duckduckgo_full_search)

# Example test
response = search_tool("Who's the current General Manager of Golden State Warriors?")
print(response.raw_output[-1]['body'])

`langgraph`

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

# Example test
results = search_tool.invoke("Who's the current General Manager of Golden State Warriors?")
print(results)

### Creating a custom tool for weather information

`smolagents`

In [ ]:
from smolagents import Tool
import random


class WeatherInfoTool(Tool):
    name: "weather_info"
    description = "Fetches dummy weather information for a given location."
    inputs = {
        'location': {
            'type': 'string',
            'description': 'The location to get weather information for.'
        }
    }
    output_type = 'string'

    def forward(self, location: str):
        # Dummyh weather data
        weather_conditions = [
            {'condition': 'Rainy', 'temp_c': 15},
            {'condition': 'Clear', 'temp_c': 25},
            {'condition': 'Windy', 'temp_c': 20}
        ]

        # Randomly select a weather condition
        data = random.choice(weather_conditions)
        return f"Weather in {location}: {data['condition']}, {data['temp_c']} degrees Celcius"


# Initialize the tool
weather_info_tool = WeatherInfoTool()

`llama-index`

In [ ]:
import random
from llama_index.core.tools improt FunctionTool


def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']} degrees Celcius"


# Initialize the tool
weather_info_tool = FunctionTool.from_defaults(get_weather_info)

`langgraph`

In [ ]:
import random
from langchain.tools import Tool


def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']} degrees Celcius"


# Initialize the tool
weather_info_tool = Tool(
    name="get_weather_info",
    func=get_weather_info,
    description="Fetches dummy weather information for a given location."
)

### Creating a hug stats tool for influential AI builders

We can also create a tool to fetch model statistics from the HuggingFace Hub based on a username.

`smolagents`

In [ ]:
from smolagents import Tool
from huggingface_hub import list_models


class HubStatsTool(Tool):
    name: 'hub_stats'
    description = 'Fetches the most downloaded model from a specific author on the HuggingFace Hub.'
    inputs = {
        'author': {
            'type': 'string',
            'description': 'The username of the model author/organization to find models from.'
        }
    }
    output_type = 'string'

    def forward(self, author: str):
        try:
            # List models from the specified author, sorted by downloads
            models = list(list_models(
                author=author,
                sort='downloads',
                direction=-1,
                limit=1
            ))

            if models:
                model = models[0]
                return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."'
            else:
                return f"No models found for author {author}."
        except Exception as e:
            return f"Error fetching models for {author}: {str(e)}"



# Initialize the tool
hub_stats_tool = HubStatsTool()

# Example test
print(hub_stats_tool('facebook'))

`llama-index`

In [ ]:
from llama_index.core.tools import FunctionTool
from huggingface_hub import list_models


def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the HuggingFace Hub."""
    try:
        # List models from the specified author, sorted by downloads
        models = list(list_models(
            author=author,
            sort='downloads',
            direction=-1,
            limit=1
        ))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."'
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"


# Initialize the tool
hub_stats_tool = FunctionTool.from_defaults(get_hub_stats)

# Example test
print(hub_stats_tool('facebook'))

`langgraph`

In [ ]:
from langchain.tools import Tool
from huggingface_hub import list_models


def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the HuggingFace Hub."""
    try:
        # List models from the specified author, sorted by downloads
        models = list(list_models(
            author=author,
            sort='downloads',
            direction=-1,
            limit=1
        ))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."'
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"


# Initialize the tool
hub_stats_tool = Tool(
    name='get_hub_stats',
    func=get_hub_stats,
    description='Fetches the most downloaded model from a specific author on the HuggingFace Hub.'
)

# Example test
print(hub_stats_tool('facebook'))

### Integrating tools with agents

Now that we have all the tools, we can integrate them into agents.

`smolagents`

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

# Initialize the HuggingFace model
model = InferenceClientModel()

# Create agent
agent = CodeAgent(
    model=model,
    tools=[
        search_tool,
        weather_info_tool,
        hub_stats_tool
    ]
)

# Example test
response = agent.run(
    "What is Facebook and what's their most popular model?"
)
print("Agent's Response:")
print(response)

`llama-index`

In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

# Initialize the HuggingFace model
llm = HuggingFaceInferenceAPI(model_name='Qwen/Qwen2.5-Coder-32B-Instruct')

# Create agent
agent = AgentWorkflow.from_tools_or_functions(
    [
        search_tool,
        weather_info_tool,
        hub_stats_tool
    ],
    llm=llm
)

# Example test
response = await agent.run(
    "What is Facebook and what's their most popular model?"
)
print("Agent's Response:")
print(response)

`langgraph`

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import START, StateGraph
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

# Initialize the HuggingFace model
llm = HuggingFaceEndpoint(
    repo_id='Qwen/Qwen2.5-Coder-32B-Instruct',
    huggingfacehub_api=HF_TOKEN
)
chat = ChatHuggingFace(llm=llm, verbose=True)
tools = [search_tool, weather_info_tool, hub_stats_tool]
chat_with_tools = chat.bind_tools(tools)

# Generate AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

## The graph
builder = StateGraph(AgentState)
# Define nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
agent = builder.compile()


# Example test
messages = [
    HumanMessage(content="What is Facebook and what's their most popular model?")
]
response = agent.invoke({'messages': messages})
print("Agent's Response:")
print(response['messages'][-1].content)

## Creating Agent

Now that we have built all the necessary components for our agents, it is time to bring everything together into a complete agent.

### Assembling agent

`smolagents`

In [ ]:
import random
from smolagents import CodeAgent, InferenceClientModel

# Import our custom tools from previous secitons
# Assume they are saved under `tools.py` and `retriever.py`
from tools import DuckDuckGoSearchTool, WeatherInfoTool, HubStatsTool
from retriever import guest_info_tool

In [ ]:
# Initialize the HuggingFace model
model = InferenceClientModel()

# Initialize the web search tool
search_tool = DuckDuckGoSearchTool()

# Initialize the weather tool
weather_info_tool = WeatherInfoTool()

# Initialize the Hub stats tool
hub_stats_tool = HubStatsTool()

# Create agent with all the tools
agent = CodeAgent(
    model=model,
    tools=[
        guest_info_tool,
        weather_info_tool,
        search_tool,
        hub_stats_tool
    ],
    add_base_tools=True,  # add any additional base tools
    planning_intervals=3, # enable planning every 3 steps
)

`llama-index`

In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

from tools import search_tool, weather_info_tool, hub_stats_tool
from retriever import guest_info_tool

In [ ]:
# Initialize the HuggingFace model
llm = HuggingFaceInferenceAPI(model_name='Qwen/Qwen2.5-COder-32B-Instruct')

# Create agent with all the tools
agent = AgentWorkflow.from_tools_or_functions(
    [
        guest_info_tool,
        weather_info_tool,
        search_tool,
        hub_stats_tool
    ],
    llm=llm
)

`langgraph`

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import START, StateGraph
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

from tools import DuckDuckGoSearchRun, weather_info_tool, hub_stats_tool
from retriever import guest_info_tool

In [ ]:
# Initialize the web search tool
search_tool = DuckDuckGoSearchRun()

# Initialize the HuggingFace model
llm = HuggingFaceEndpoint(
    repo_id='Qwen/Qwen2.5-Coder-32B-Instruct',
    huggingfacehub_api=HF_TOKEN
)
chat = ChatHuggingFace(llm=llm, verbose=True)
tools = [search_tool, weather_info_tool, hub_stats_tool, guest_info__tool]
chat_with_tools = chat.bind_tools(tools)

# Generate AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }


# The graph
builder = StateGraph(AgentState)
# Define nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
agent = builder.compile()

### End-to-end examples

#### Example 1: Finding guest information

`smolagents`

In [ ]:
query = "Tell me about 'Lady Ada Lovelace'"
response = agent.run(query)

print("Agent's Response:")
print(response)

`llama-index`

In [ ]:
query = "Tell me about 'Lady Ada Lovelace'"
response = await agent.run(query)

print("Agent's Response:")
print(response.response.blocks[0].text)

`langgraph`

In [ ]:
query = "Tell me about 'Lady Ada Lovelace'"
messages = {"messages": query}
response = agent.invoke(messages)

print("Agent's Response:")
print(response['messages'][-1].content)

#### Example 2: Checking the weather

`smolagents`

In [ ]:
query = "What's the weather like in Houston tonight? Will it be suitable for our fireworks display?"
response = agent.run(query)

print("Agent's Response:")
print(response)

`llama-index`

In [ ]:
query = "What's the weather like in Houston tonight? Will it be suitable for our fireworks display?"
response = await agent.run(query)

print("Agent's Response:")
print(response.response.blocks[0].text)

`langgraph`

In [ ]:
query = "What's the weather like in Houston tonight? Will it be suitable for our fireworks display?"
messages = {"messages": query}
response = agent.invoke(messages)

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

#### Example 3: Impressing AI reseaerchers

`smolagents`

In [ ]:
query = "One of our guests is from Qwen. What can you tell me about their most popular model?"
response = agent.run(query)

print("Agent's Response:")
print(response)

`llama-index`

In [ ]:
query = "One of our guests is from Qwen. What can you tell me about their most popular model?"
response = await agent.run(query)

print("Agent's Response:")
print(response.response.blocks[0].text)

`langgraph`

In [ ]:
query = "One of our guests is from Qwen. What can you tell me about their most popular model?"
messages = {"messages": query}
response = agent.invoke(messages)

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

#### Example 4: Combining multiple tools

`smolagents`

In [ ]:
query = "I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for this conversation?"
response = agent.run(query)

print("Agent's Response:")
print(response)

`llama-index`

In [ ]:
query = "I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for this conversation?"
response = await agent.run(query)

print("Agent's Response:")
print(response.response.blocks[0].text)

`langgraph`

In [ ]:
query = "I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for this conversation?"
messages = {"messages": query}
response = agent.invoke(messages)

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

### Advanced features: conversation memory

`smolagents`

In [ ]:
# Create agent with conversation memory
agent_with_memory = CodeAgent(
    model=model,
    tools=[
        guest_info_tool,
        weather_info_tool,
        search_tool,
        hub_stats_tool
    ],
    add_base_tools=True,
    planning_intervals=3,
)

# First interaction
response1 = agent_with_memory.run(
    "Tell me about Lady Ada Lovelace."
)
print("Agent's First Response:")
print(response1)

# Second interaction (referencing the first)
response2 = agent_with_memory.run(
    "What projects is she currently working on?",
    reset=False     # IMPORTANT
)
print("Agent's Second Response:")
print(response2)

`llama-index`

In [ ]:
from llama_index.core.workflow import Context

agent = AgentWorkflow.from_tools_or_functions(
    [
        guest_info_tool,
        weather_info_tool,
        search_tool,
        hub_stats_tool
    ],
    llm=llm
)

# Remembering state (IMPORTANT)
ctx = Context(agent)

# First interaction
response1 = await agent.run(
    "Tell me about Lady Ada Lovelace.",
    ctx=ctx
)
print("Agent's First Response:")
print(response1)

# Second interaction (referencing the first)
response2 = await agent.run(
    "What projects is she currently working on?",
    ctx=ctx"
)
print("Agent's Second Response:")
print(response2)

`langgraph`

In [ ]:
# First interaction
response = agent.invoke(
    {'messages': [
        HumanMessage(content="Tell me about Lady Ada Lovelace.")
    ]}
)
print("Agent's First Response:")
print(response['messages'][-1].content)
print()

# Second interaction (referencing the first)
response = agent.invoke(
    {'messages':
        response['messages'] + [
            HumanMessage(content="What projects is she currently working on?")
        ]
    }
)
print("Agent's Second Response:")
print(response['messages'][-1].content)